In [42]:
from pyedflib import highlevel
import os
from os import listdir
from os.path import isfile, join
from pandas import read_csv
from sklearn.ensemble import RandomForestRegressor as RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score, mean_squared_error
import numpy as np

In [26]:
et_arithmetic_directory = "../datasets/jetris"

In [3]:
dataset = read_csv(join(et_arithmetic_directory, "ALLCombinedVariables_timeseriesPupilEvolution.csv"))

In [4]:
# (24,21,31000)
# (24, 31000)
# (24, 50)
# reshape to only use measurements from 1 electrode, and 50 samplings
unique_game_ids = dataset["gameID"].unique()
list_of_dataframes_on_id = [dataset.loc[dataset['gameID'] == game_id] for game_id in unique_game_ids]

In [27]:
labels = [df.iloc[-1]["Score.1"] for df in list_of_dataframes_on_id]
mapSaccade = lambda x: float(x) if x!="saccade" else 69.0
pupil_change_on_gameID = [list(map(mapSaccade, df["Pupil.change"].tolist())) for df in list_of_dataframes_on_id]

In [28]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

def get_moving_averages(x, N):
    step = int(len(x)/N - 0.5)
    averages = []
    for i in range(0,len(x),step):
        try:
            sublist = x[i:i+step]
            average = sum(sublist)/len(sublist)
            averages.append(average)
        except IndexError:
            pass
    return averages
        
        
        


In [28]:
print(pupil_change_on_gameID[0])

[1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367, 1.4544512137220367,

In [61]:
# Train test split
x = [get_moving_averages(lst, 100) for lst in pupil_change_on_gameID]
x_train, x_test, y_train, y_test = train_test_split(
    x, labels, test_size=0.33, random_state=89
)
from sklearn import linear_model
model = linear_model.Lasso(alpha=0.1)
model.fit(x_train, y_train)

/home/august/.cache/pypoetry/virtualenvs/ideal-pancake-2nvydFT1-py3.8/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2254279.9770968957, tolerance: 13133.476635514022
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [62]:
y_pred = model.predict(x_test)

In [63]:
print(y_pred)
print(np.array(y_test))

[ 2860.91459965   792.99228347 -3429.31624812  3503.27190917
   543.69375663 -1728.11649859   724.96901083   129.61500159
 -4073.95178212  4338.64986293  4427.47279601  4881.11946746
 -1727.46630938  3822.8513815   -928.56939641 -2104.58127066
 -1652.28434272 11441.35904278 -2595.6007723  -2460.2634299
  4457.7782788  -1969.98086697 -2371.88442755  2296.04418475
  2104.38108387  2769.10549171   857.23736945  4191.7658887
  3699.75167462  3451.7261811  -2461.37391989 -3590.77295734
 -2504.28316691 -3676.70064794  7381.59761953 -2680.9581467
   985.07463818   913.07858336  2330.72857128  3139.3566968
  1919.20067762  3343.63219389  3186.50363946 -3693.20198331
 -9438.16230168 -2871.70081352   -63.85530114   876.31660531
   258.73045442  -941.08898985  1764.1553511   2472.49006619
  8125.37779344]
[2900  300 3700  600    0 1100  500 2700  400 3400  300  300 2200  100
 3900  100  400 1200  200 2400  700 4900 1300    0 2100 1700 1000 2200
  500 1100  900  400  100  200 2400  200 1800 3700  

In [64]:
print(mean_absolute_error(y_test, [int(x) for x in y_pred]))
print([int(x) for x in y_pred])
print(np.array(y_test))

3006.754716981132
[2860, 792, -3429, 3503, 543, -1728, 724, 129, -4073, 4338, 4427, 4881, -1727, 3822, -928, -2104, -1652, 11441, -2595, -2460, 4457, -1969, -2371, 2296, 2104, 2769, 857, 4191, 3699, 3451, -2461, -3590, -2504, -3676, 7381, -2680, 985, 913, 2330, 3139, 1919, 3343, 3186, -3693, -9438, -2871, -63, 876, 258, -941, 1764, 2472, 8125]
[2900  300 3700  600    0 1100  500 2700  400 3400  300  300 2200  100
 3900  100  400 1200  200 2400  700 4900 1300    0 2100 1700 1000 2200
  500 1100  900  400  100  200 2400  200 1800 3700  400  900 1100 3400
 1500 1000  800  300  600 1100 4000  900 3600  100  400]
